In [19]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ximo/GitHub/skforecast


In [20]:
import keras
import numpy as np
import pandas as pd

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model, create_and_compile_model_exog

# set backend torch
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
print(keras.__version__)
print(keras.backend.backend())

series = pd.DataFrame(
    {
        "1": np.arange(50.),
        "2": np.arange(50., 100.),
        "3": np.arange(100., 150.),
    }
)

exog = pd.DataFrame(
    {
        "exog_1": np.arange(50.),
        "exog_2": np.arange(50., 100.),
        "exog_3": np.arange(100., 150.),
    }
)

exog_predict = pd.DataFrame(
    {
        "exog_1": np.arange(50., 60),
        "exog_2": np.arange(100., 110.),
        "exog_3": np.arange(150., 160.),
    },
    index=pd.RangeIndex(start=50, stop=60, step=1)
)

3.9.2
tensorflow


In [21]:
lags = 4
steps = 5
levels = ["1", "2"]
activation = "relu"
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]


# model = create_and_compile_model(
#     series=series,
#     lags=lags,
#     steps=steps,
#     levels=levels,
#     recurrent_units=recurrent_units,
#     dense_units=dense_units,
#     activation=activation,
#     optimizer=optimizer,
#     loss=loss,
# )

model = create_and_compile_model_exog(
    series=series,
    lags=lags,
    steps=steps,
    levels=levels,
    exog=exog,
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 4, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 100)       │     41,600 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 100)       │          0 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 103)       │          0 │ flatten_1[0][0],  │
│ (Concatenate)       │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     13,312 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 10)        │        650 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 5, 2)      │          0 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 63,818 (249.29 KB)

 Trainable params: 63,818 (249.29 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
forecaster = ForecasterRnn(model, levels, lags=lags, transformer_series=None, transformer_exog=None)
forecaster

============= 
ForecasterRnn 
============= 
Regressor: <Functional name=functional_1, built=True> 
Target series (levels): ['1', '2'] 
Lags: [1 2 3 4] 
Window size: 4 
Maximum steps to predict: [1 2 3 4 5] 
Multivariate series: None 
Exogenous included: False 
Exogenous names: None 
Transformer for series: None 
Transformer for exog: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'name': 'functional_1', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': (None, 4, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer_2'}, 'registered_name': None, 'name': 'input_layer_2', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'LSTM', 'config': {'name': 'lstm_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'zero_output_for_mask': False, 'units': 100, 'activation': 'relu', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'recurrent_initializer': {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'seed': None, 'gain': 1.0}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'seed': None}, 'registered_name': None, 'build_config': {'input_shape': [None, 4, 3]}, 'name': 'lstm_1', 'inbound_nodes': [{'args': ({'class_name': '__keras_tensor__', 'config': {'shape': (None, 4, 3), 'dtype': 'float32', 'keras_history': ['input_layer_2', 0, 0]}},), 'kwargs': {'training': False, 'mask': None}}]}, {'module': 'keras.layers', 'class_name': 'Flatten', 'config': {'name': 'flatten_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'data_format': 'channels_last'}, 'registered_name': None, 'build_config': {'input_shape': [None, 100]}, 'name': 'flatten_1', 'inbound_nodes': [{'args': ({'class_name': '__keras_tensor__', 'config': {'shape': (None, 100), 'dtype': 'float32', 'keras_history': ['lstm_1', 0, 0]}},), 'kwargs': {}}]}, {'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': (None, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer_3'}, 'registered_name': None, 'name': 'input_layer_3', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Concatenate', 'config': {'name': 'concatenate_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'axis': -1}, 'registered_name': None, 'build_config': {'input_shape': [[None, 100], [None, 3]]}, 'name': 'concatenate_1', 'inbound_nodes': [{'args': ([{'class_name': '__keras_tensor__', 'config': {'shape': (None, 100), 'dtype': 'float32', 'keras_history': ['flatten_1', 0, 0]}}, {'class_name': '__keras_tensor__', 'config': {'shape': (None, 3), 'dtype': 'float32', 'keras_history': ['input_layer_3', 0, 0]}}],), 'kwargs': {}}]}, {'module': 'keras.layers', 'class_name': 'Dense', 'config': {'name': 'dense_3', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias

In [23]:
# forecaster.fit(
#     series=series, store_in_sample_residuals=True
# )

forecaster.fit(
    series=series, exog=exog, store_in_sample_residuals=True
)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2643.5046 


In [24]:
forecaster.series_names_in_

['1', '2', '3']

In [25]:
forecaster.levels

['1', '2']

In [26]:
forecaster.last_window_

,1,2,3
46,46.0,96.0,146.0
47,47.0,97.0,147.0
48,48.0,98.0,148.0
49,49.0,99.0,149.0


In [27]:
# inputs = forecaster._create_predict_inputs(
#     steps=3, levels=['1', '2']
# )


X, X_col_names, steps, levels, prediction_index = forecaster._create_predict_inputs(
    steps=5, levels=['1', '2'], exog=exog_predict
)
print(X[0])
print(X[1])
print(X_col_names)
print(steps)
print(levels)
print(prediction_index)

[[[ 46.  96. 146.]
  [ 47.  97. 147.]
  [ 48.  98. 148.]
  [ 49.  99. 149.]]]
[[ 51. 101. 151.]
 [ 52. 102. 152.]
 [ 53. 103. 153.]
 [ 54. 104. 154.]
 [ 55. 105. 155.]]
['exog_1', 'exog_2', 'exog_3']
[1, 2, 3, 4, 5]
['1', '2']
Index([50, 51, 52, 53, 54], dtype='int64')


In [31]:
X[0][0]

array([[ 46.,  96., 146.],
       [ 47.,  97., 147.],
       [ 48.,  98., 148.],
       [ 49.,  99., 149.]])

In [32]:
X[1]

array([[ 51., 101., 151.],
       [ 52., 102., 152.],
       [ 53., 103., 153.],
       [ 54., 104., 154.],
       [ 55., 105., 155.]])

In [30]:
forecaster.regressor.predict(X)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1, 5


In [ ]:
# predictions = forecaster.predict(steps=2, levels=['1', "2"])
# predictions

predictions = forecaster.predict(steps=5, exog=exog_predict, levels=['1', "2"])
predictions

[array([[[0.93877551, 0.93877551, 0.93877551],
        [0.95918367, 0.95918367, 0.95918367],
        [0.97959184, 0.97959184, 0.97959184],
        [1.        , 1.        , 1.        ]]]), array([[1.14634146, 1.14634146, 1.14634146],
       [1.17073171, 1.17073171, 1.17073171],
       [1.19512195, 1.19512195, 1.19512195],
       [1.2195122 , 1.2195122 , 1.2195122 ],
       [1.24390244, 1.24390244, 1.24390244]])]


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1, 5


In [ ]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=None)
predictions

,level,pred,lower_bound,upper_bound
50,1,24.082923,2.338655,45.827191
50,2,67.324215,39.675404,94.973027
51,1,8.498265,-27.938370,44.934899
51,2,72.027273,47.464998,96.589547
52,1,12.400389,-21.560663,46.361441
52,2,36.669044,-19.810917,93.149005


In [ ]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=['1'])
predictions

,level,pred,lower_bound,upper_bound
50,1,24.082923,2.338655,45.827191
51,1,8.498265,-27.938370,44.934899
52,1,12.400389,-21.560663,46.361441
